In [25]:
import numpy as np
import pandas as pd

# Enoncé

Créer une class StandardScaler2 avec comme méthode fit, transform, fit_transform et en attribut un dicitonnaire params_ qui contient la moyenne et l'écart_type de vos données. Les données en entrées sont uniquement de array numpy (pas des dataframes)
me dire que vous avez terminé avant de faire le bonus.
en bonus :
- mettre une erreur si on essaye de transform sans avoir de fit
- permettre au code de fonctionner, peut importe si l'entrée est une numpy array ou une dataframe

La classe désirée s'inspire grandement de [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) sur Scikit-Learn.

La standardisation `z` d'un score `x` se calcule avec la formule suivante :

$$
z = \frac{x-u}{s}
$$

Avec

* `u` la moyenne des échantillons d'entraînement si `with_mean=True`, 0 sinon
* `s` l'écart-type des échantillons d'entraînement si `with_std=True`, 0 sinon

## Démonstration sur Scikit-Learn

In [4]:
from sklearn.preprocessing import StandardScaler
data = [[0, 0], [0, 0], [1, 1], [1, 1]]
scaler = StandardScaler()



In [7]:
print("Méthode fit: ", scaler.fit(data))
print("Moyenne: ", scaler.mean_)
print("Transformation: ")
print(scaler.transform(data))
print("Transformation sur un vecteur (2,2)",scaler.transform([[2, 2]]))

Méthode fit:  StandardScaler(copy=True, with_mean=True, with_std=True)
Moyenne:  [0.5 0.5]
Transformation: 
[[-1. -1.]
 [-1. -1.]
 [ 1.  1.]
 [ 1.  1.]]
Transformation sur un vecteur (2,2) [[3. 3.]]


## Création de la méthode

In [23]:
class StandardScaler2:
    def __init__(self):
        self.params_ = {}
    def fit(self, x):
        self.params_['mean'] = np.mean(x)
        self.params_['std'] = np.std(x)
        return self
    def fit_transform(self, x):
        return (x-self.params_['mean'])/self.params_['std']

new_scaler = StandardScaler2()

In [24]:
data = np.array(data)
print("Méthode fit: ", new_scaler.fit(data))
print("Moyenne: ", new_scaler.params_)
print("Transformation: ")
print(new_scaler.fit_transform(data))
print("Transformation sur un vecteur (2,2)",new_scaler.fit_transform([[2, 2]]))

Méthode fit:  <__main__.StandardScaler2 object at 0x000001CF74DFF048>
Moyenne:  {'mean': 0.5, 'std': 0.5}
Transformation: 
[[-1. -1.]
 [-1. -1.]
 [ 1.  1.]
 [ 1.  1.]]
Transformation sur un vecteur (2,2) [[3. 3.]]


## Question bonus

In [35]:
df = pd.read_csv('wine.csv')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


### Affichage d'erreur

In [36]:
class StandardScaler2:
    def __init__(self):
        self.params_ = {}
    def fit(self, x):
        self.params_['mean'] = np.mean(x)
        self.params_['std'] = np.std(x)
        return self
    def fit_transform(self, x):
        try:
            return (x-self.params_['mean'])/self.params_['std']
        except:
            print("Error: must perform StandardScaler2.fit(x) first.")

In [37]:
new_scaler = StandardScaler2()
print(new_scaler.fit_transform(np.array(df['free sulfur dioxide'])))

Error: must perform StandardScaler2.fit(x) first.
None


### Gérer les dataframe

In [80]:
class StandardScaler2:
    def __init__(self):
        self.params_ = {}
    def fit(self, x):
        x = self.check_type(x)
        self.params_['mean'] = np.mean(x,axis=0)
        self.params_['std'] = np.std(x,axis=0)
        return self
    def fit_transform(self, x):
        x = self.check_type(x)
        try:
            return (x-np.tile(self.params_['mean'],(x.shape[0],1)))/np.tile(self.params_['std'],(x.shape[0],1))
        except:
            print("Error: must perform StandardScaler2.fit(x) first.")
    def check_type(self,x):
        if 'DataFrame' in str(type(x)):
            x = np.array(x)
        return x

In [82]:
new_scaler = StandardScaler2()
new_scaler.fit(df)
new_scaler.fit_transform(df)

array([[-0.52835961,  0.96187667, -1.39147228, ..., -0.57920652,
        -0.96024611, -0.78782264],
       [-0.29854743,  1.96744245, -1.39147228, ...,  0.1289504 ,
        -0.58477711, -0.78782264],
       [-0.29854743,  1.29706527, -1.18607043, ..., -0.04808883,
        -0.58477711, -0.78782264],
       ...,
       [-1.1603431 , -0.09955388, -0.72391627, ...,  0.54204194,
         0.54162988,  0.45084835],
       [-1.39015528,  0.65462046, -0.77526673, ...,  0.30598963,
        -0.20930812, -0.78782264],
       [-1.33270223, -1.21684919,  1.02199944, ...,  0.01092425,
         0.54162988,  0.45084835]])

In [79]:
np.tile(new_scaler.params_['mean'],(1599,1)).shape

(1599, 12)